In [1]:
import requests
import sqlite3
from bs4 import BeautifulSoup
import os
import pandas as pd
from fake_useragent import UserAgent
import time

In [2]:
# Creación de la base de datos
con = sqlite3.connect('anagrama.db')
cursorANA = con.cursor()
cursorANA.execute(
    '''
    CREATE TABLE IF NOT EXISTS anagrama
    (
    isbn VARCHAR,
    ean INT,
    código VARCHAR,
    título TEXT,
    autor TEXT,
    precio FLOAT,
    páginas INT,
    fecha_publicación DATE,
    colección TEXT,
    traducción TEXT,
    sinopsis TEXT,
    ebook BOOLEAN INT
    );
    ''')
con.commit()
con.close()

In [ ]:
# Creación de un directorio para almacenar las imágenes
if not os.path.exists('portadas'):
    os.mkdir('portadas')

In [7]:
# Esqueleto del scraper

def scrap_anagrama(proportional_delay=True):
    
    def get_links_colecciones():
        '''
        Devuelve diccionario que contiene las colecciones
        de la editorial Anagrama
        '''
        html = requests.get(url_colecciones, headers).content
        soup = BeautifulSoup(html)
        colecciones = {}
        for link in soup.findAll('a'):
            coleccion_link = url_anagrama + str(link.get('href'))
            coleccion = coleccion_link.rsplit('/', 1)[-1].replace('-',' ')
            if "/coleccion/" in coleccion_link:
                colecciones[coleccion] = coleccion_link
        del colecciones['ebooks']
        return colecciones


    def get_links_libros_in_page(url_page):
        '''
        Devuelve una lista que contiene los libros
        de una página del sitio web
        '''
        html = requests.get(url_page, headers).content
        soup = BeautifulSoup(html)
        links_libros = set([link['href'] for link in soup.find_all(
            "a", class_="book", href=True) if link['href'].startswith('/libro/')])
        return links_libros
        

    def get_libro_info(url_libro):
        '''
        Recoge la información básica del libro
        dada una url de la editorial Anagrama
        y lo carga en un diccionario
        '''
        url_libro = url_libro
        html = requests.get(url_libro, headers).content
        soup = BeautifulSoup(html)
        
        datos_libro = dict()
        
        # Básicos
        datos_libro["TÍTULO"] = soup.h1.get_text()
        datos_libro["AUTOR/A"] = soup.title.get_text().split('-')[1].strip()
        datos_libro["RESUMEN"] = soup.find('div', class_="textContent mt10px").get_text()
        
        # Tabla
        for e in soup.find('table'):
            data = e.find_all('td')
            col, row = data[0].get_text(), data[1].get_text()
            datos_libro[col] = row
        
        # Ebook 
        datos_libro["EBOOK"] = soup.find("div", class_="tab-pane ebook") != None
        
        # Corrección de la información de COLECCIÓN
        datos_libro["COLECCIÓN"] = url_libro.split('/')[4].replace('-',' ').title()
        
        # Portada libro
        try: 
            datos_libro["URL PORTADA"] = soup.find('meta', property="og:image")['content']
        except:
            pass

        return datos_libro
        

    def get_row_values(info_libro):
        '''
        Devuelve una lista de la información de un libro en formato apto
        para ser cargado en la base de datos
        '''
        columns = ['ISBN', 'EAN', 'TÍTULO', 'AUTOR/A', 'COLECCIÓN', 'CÓDIGO',
                   'NÚM. DE PÁGINAS', 'PUBLICACIÓN', 'PVP CON IVA', 'RESUMEN',
                   'TRADUCCIÓN', 'EBOOK']
        for key, value in info_libro.items():
            row_values = []
            for column in columns:
                try:
                    row_values.append(info_libro[column])
                except:
                    row_values.append(None)
        return row_values
        

    def sql_insert(con, row_values):
        '''
        Actualiza la base de datos con un nuevo registro
        '''
        cursorObj = con.cursor()
        cursorObj.execute('''
                INSERT INTO 
                anagrama(isbn, ean, título, autor, colección, código,
                páginas, fecha_publicación, precio, sinopsis, traducción, ebook
                ) 
                VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', row_values)
        con.commit()
        
    
    def load_requests(source_url, ean, coleccion):
        '''
        Descarga y almacena la portada del libro
        '''
        r = requests.get(source_url, headers=headers, stream=True)
        if r.status_code == 200:
            ruta = "portadas\\"+ean+'-'+coleccion+'.jpg'
            output = open(ruta,"wb")
            for chunk in r:
                output.write(chunk)
            output.close()
    
    
    con = sqlite3.connect('anagrama.db')

    url_anagrama = 'https://www.anagrama-ed.es'
    url_colecciones = 'https://www.anagrama-ed.es/colecciones'
    
    headers = {"User_Agent":UserAgent().random}  # User agent generado aleatoriamente
    
    colecciones = get_links_colecciones()
    
    for coleccion, link in colecciones.items():
        page = 1
        while bool(page)==True:
            url_page = link + '?p=' + str(page)
            listado_links_libros = get_links_libros_in_page(url_page)
            if len(listado_links_libros) != 0:
                for url_libro in listado_links_libros:
                    url_libro = url_anagrama + url_libro
                    t0 = time.time()
                    info_libro = get_libro_info(url_libro)
                    response_delay = time.time() - t0
                    row_values = get_row_values(info_libro)
                    sql_insert(con, row_values)
                    try:
                        load_requests(info_libro['URL PORTADA'],
                                     info_libro['EAN'],
                                     info_libro['COLECCIÓN'])
                    except: 
                        pass
                    if proportional_delay==True: 
                        time.sleep(10*response_delay)  # Espaciado automático entre peticiones
                page += 1
            else:
                page = False
    return '¡Listo!'

In [5]:
# Raspado del sitio web y generación de informe

scrap_anagrama(False)

'¡Listo!'

In [6]:
# Conversión de la base de datos a csv

conn = sqlite3.connect('anagrama.db', isolation_level=None,
                      detect_types=sqlite3.PARSE_COLNAMES)

df = pd.read_sql_query("SELECT * FROM anagrama", conn)
df.to_csv('database.csv', index=False)